<a href="https://colab.research.google.com/github/Andron00e/CBM_Optim/blob/main/llama70Bprompts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q git+https://github.com/huggingface/peft.git git+https://github.com/huggingface/transformers.git
!pip install accelerate
!pip install bitsandbytes
!pip install sentencepiece
!pip install datasets

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import torch
torch.cuda.is_available()

True

In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch.nn as nn
import bitsandbytes as bnb

Llama-2-70B-chat-GPTQ

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "TheBloke/Llama-2-70B-chat-GPTQ",
    torch_dtype = torch.float16,
    load_in_8bit=True,
    device_map = "auto"
)
tokenizer = AutoTokenizer.from_pretrained("TheBloke/Llama-2-70B-chat-GPTQ")
tokenizer.pad_token_id = (
    0
)
tokenizer.padding_side = "left"

print(model)

In [ ]:
from transformers import pipeline

pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=AutoTokenizer.from_pretrained("TheBloke/Llama-2-70B-chat-GPTQ"),
    torch_dtype=torch.float16,
    device_map="auto",
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=50
)

In [ ]:
def generation_function(prompt: str):
  features = pipeline(prompt)[0]["generated_text"][len(prompt):]
  features = features.split("\n-")
  features = [feat.replace("\n", "") for feat in features]
  features = [feat.strip()for feat in features]
  features = [feat for feat in features if len(feat)>0]
  features = set(features)
  return features

In [ ]:
import json

generated_answers = {}
output_file = "data/concepts_llama.json"
save_classes = "data/all_classes.txt"
with open(save_classes, "r") as f:
  classes = f.read().lower().split("\n")


for label in classes:
  generated_answers[label] = {}

  prompt1 = f"List the most important features for recognizing something as a {label}? Write them one by one."
  generated1 = generation_function(prompt1)
  generated_answers[label]["A1"] = list(generated1)

  prompt2 = f"List the things most commonly seen around a {label}. Write them one by one."
  generated2 = generation_function(prompt2)
  generated_answers[label]["A2"] = list(generated2)

  prompt3 = f"Give a generalization for the word {label}"
  generated3 = generation_function(prompt3)
  generated_answers[label]["A3"] = list(generated3)


with open(output_file, "w") as json_file:
    json.dump(generated_answers, json_file, indent=4)

TheBloke/llama-2-70b-Guanaco-QLoRA-fp16

**System:**
{prompt}

**Assistant:**

TheBloke/Llama-2-70B-GGML


**System:**
{system_message}

**User:**
{prompt}

**Assistant:**
